In [1]:
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.33)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

In [2]:
# keras 或 tensorflow 调用GPU报错：Blas GEMM launch failed
# https://blog.csdn.net/Leo_Xu06/article/details/82023330
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 

Using TensorFlow backend.


In [3]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


C:\Users\lee\Anaconda3\envs\shu\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\lee\Anaconda3\envs\shu\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [4]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 262us/step - loss: 0.4801 - acc: 0.7956
Epoch 2/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4284 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4235 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4188 - acc: 0.8090
Epoch 5/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4164 - acc: 0.8241
Epoch 6/100
8000/8000 [==============================] - 2s 206us/step - loss: 0.4146 - acc: 0.8277
Epoch 7/100
8000/8000 [==============================] - 2s 203us/step - loss: 0.4131 - acc: 0.8316
Epoch 8/100
8000/8000 [==============================] - 2s 202us/step - loss: 0.4118 - acc: 0.8321
Epoch 9/100
8000/8000 [==============================] - 2s 209us/step - loss: 0.4107 - acc: 0.8329
Epoch 10/100
8000/8000 [==============================] - 2s 203us/step - loss: 0.4100 - acc: 0.8337

8000/8000 [==============================] - 2s 205us/step - loss: 0.4008 - acc: 0.8350
Epoch 83/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4011 - acc: 0.8361
Epoch 84/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4009 - acc: 0.8337
Epoch 85/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4009 - acc: 0.8342
Epoch 86/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4009 - acc: 0.8350
Epoch 87/100
8000/8000 [==============================] - 2s 205us/step - loss: 0.4008 - acc: 0.8340
Epoch 88/100
8000/8000 [==============================] - 2s 203us/step - loss: 0.4010 - acc: 0.8331
Epoch 89/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4011 - acc: 0.8352
Epoch 90/100
8000/8000 [==============================] - 2s 203us/step - loss: 0.4010 - acc: 0.8351
Epoch 91/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4008 - acc: 0.8345
Epo

In [5]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)